In [19]:
import csv
import pandas as pd
import folium
import json

In [2]:
pd.set_option("max_colwidth", None)

### Step 1. Prepare Township Ranges along with their Bounding Box

In [9]:
tr_bbox = []
with open('township_ranges_bigten.csv') as f:
    reader = csv.reader(f)
    fields = next(reader)
    for row in reader:
        state = row[0]
        if state == 'MN':
            township_code = str(row[3])
            township_dir = row[4]
            range_code = str(row[5])
            range_dir = row[6]
            bbox = row[-1].replace(' ', '')   # remove whitespaces
            tr_bbox.append([township_code, township_dir, range_code, range_dir, bbox])

df = pd.DataFrame(tr_bbox, columns = ['Township Code', 'Township Direction', 'Range Code', 'Range Direction', 'Bounding Box'])

In [10]:
# convert bounding box format
df = pd.concat([df, df['Bounding Box'].str.split(',', expand=True).astype(float).round(2)], axis=1).rename(
    columns={0:'minX', 1:'minY', 2:'maxX', 3:'maxY'})
df['maxXmaxY'] = df.apply(lambda row: [row.maxX, row.maxY], axis = 1)
df['maxXminY'] = df.apply(lambda row: [row.maxX, row.minY], axis = 1)
df['minXminY'] = df.apply(lambda row: [row.minX, row.minY], axis = 1)
df['minXmaxY'] = df.apply(lambda row: [row.minX, row.maxY], axis = 1)
df['coordinates'] = df[['maxXmaxY', 'maxXminY', 'minXminY', 'minXmaxY', 'maxXmaxY']].values.tolist()
df = df.drop(columns =['minX', 'minY', 'maxX', 'maxY', 'maxXmaxY', 'maxXminY', 'minXminY', 'minXmaxY', 'Bounding Box'])

In [11]:
df.shape

(2548, 5)

### Step 2. Create GeoJSON features

In [12]:
def create_geojson_features(df):
    features = []
    geojson = {
        'type': 'FeatureCollection',
        'title': 'MN Plats Index Map',
        'features': features
    }
    for _, row in df.iterrows():
        
        coord = [row['coordinates']]
        title = f"T{row['Township Code'].zfill(3)}{row['Township Direction']}R{row['Range Code'].zfill(2)}{row['Range Direction']}"
        search_q = f"T{row['Township Code'].zfill(3)}{row['Township Direction']}+R{row['Range Code'].zfill(2)}{row['Range Direction']}"
        digHold = 'https://geo.btaa.org/?search_field=all_fields&q=' + search_q

        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Polygon', 
                'coordinates':coord
            },
            'properties': {
                'title': title,
                'digHold': digHold
            }
        }

        features.append(feature)
    return geojson

In [20]:
geojson = create_geojson_features(df)

with open('mn_township_index_map.geojson', 'w') as f:
    json.dump(geojson, f)
    

> Creating GeoJSON features...


In [18]:


m = folium.Map(location = [42.3756, -93.6397], control_scale = True, zoom_start = 7)
folium.GeoJson(geojson,show = True, tooltip = folium.GeoJsonTooltip(fields=('title', 'digHold'),
                                               aliases=('title','digHold'))).add_to(m)


m

> Creating GeoJSON features...
